In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import matplotlib as mpl
from cartopy.util import add_cyclic_point
import cmaps
import sacpy as scp
import matplotlib.path as mpath  
import sys
sys.path.append("/public/home/songqh/Documents/python_packages")
from waf import tnwaf
from sqh_toolbox import plot_figure
import matplotlib as mpl
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
filec_6 = xr.open_dataset("/data1/songqh/CESM_result/CAM5_CTL_RUN_f09g16/CAM5_CTL_RUN_f09g16.cam.h0.all-06.nc",decode_times=False)
fileh_6 = xr.open_dataset("/data1/songqh/CESM_result/ISM_heating_f09g16/ISM_heating_f09g16.cam.h0.all-06.nc",decode_times=False)
filec_7 = xr.open_dataset("/data1/songqh/CESM_result/CAM5_CTL_RUN_f09g16/CAM5_CTL_RUN_f09g16.cam.h0.all-07.nc",decode_times=False)
fileh_7 = xr.open_dataset("/data1/songqh/CESM_result/ISM_heating_f09g16/ISM_heating_f09g16.cam.h0.all-07.nc",decode_times=False)
filec_8 = xr.open_dataset("/data1/songqh/CESM_result/CAM5_CTL_RUN_f09g16/CAM5_CTL_RUN_f09g16.cam.h0.all-08.nc",decode_times=False)
fileh_8 = xr.open_dataset("/data1/songqh/CESM_result/ISM_heating_f09g16/ISM_heating_f09g16.cam.h0.all-08.nc",decode_times=False)

In [ ]:
lon = filec_6['lon']
lat = filec_6['lat']
lev = np.array(filec_6['lev'])

In [ ]:
hgtc_6 = np.array(filec_6['Z3'].loc[:,lev[12]])[5:]
hgth_6 = np.array(fileh_6['Z3'].loc[:,lev[12]])[5:]
hgtc_7 = np.array(filec_7['Z3'].loc[:,lev[12]])[5:]
hgth_7 = np.array(fileh_7['Z3'].loc[:,lev[12]])[5:]
hgtc_8 = np.array(filec_8['Z3'].loc[:,lev[12]])[5:]
hgth_8 = np.array(fileh_8['Z3'].loc[:,lev[12]])[5:]

slpc_6 = np.array(filec_6['PSL'].loc[:])[5:]
slph_6 = np.array(fileh_6['PSL'].loc[:])[5:]
slpc_7 = np.array(filec_7['PSL'].loc[:])[5:]
slph_7 = np.array(fileh_7['PSL'].loc[:])[5:]
slpc_8 = np.array(filec_8['PSL'].loc[:])[5:]
slph_8 = np.array(fileh_8['PSL'].loc[:])[5:]

airc_6 = np.array(filec_6['T'].loc[:,lev[-1]])[5:]
airh_6 = np.array(fileh_6['T'].loc[:,lev[-1]])[5:]
airc_7 = np.array(filec_7['T'].loc[:,lev[-1]])[5:]
airh_7 = np.array(fileh_7['T'].loc[:,lev[-1]])[5:]
airc_8 = np.array(filec_8['T'].loc[:,lev[-1]])[5:]
airh_8 = np.array(fileh_8['T'].loc[:,lev[-1]])[5:]

uwndc_6 = np.array(filec_6['U'].loc[:,lev[-1]])[5:]
uwndh_6 = np.array(fileh_6['U'].loc[:,lev[-1]])[5:]
uwndc_7 = np.array(filec_7['U'].loc[:,lev[-1]])[5:]
uwndh_7 = np.array(fileh_7['U'].loc[:,lev[-1]])[5:]
uwndc_8 = np.array(filec_8['U'].loc[:,lev[-1]])[5:]
uwndh_8 = np.array(fileh_8['U'].loc[:,lev[-1]])[5:]

vwndc_6 = np.array(filec_6['V'].loc[:,lev[-1]])[5:]
vwndh_6 = np.array(fileh_6['V'].loc[:,lev[-1]])[5:]
vwndc_7 = np.array(filec_7['V'].loc[:,lev[-1]])[5:]
vwndh_7 = np.array(fileh_7['V'].loc[:,lev[-1]])[5:]
vwndc_8 = np.array(filec_8['V'].loc[:,lev[-1]])[5:]
vwndh_8 = np.array(fileh_8['V'].loc[:,lev[-1]])[5:]

vwndc_ism_6 = np.array(filec_6['V'].loc[:,:,-90:80,60:100])[5:].mean(-1)
vwndh_ism_6 = np.array(fileh_6['V'].loc[:,:,-90:80,60:100])[5:].mean(-1)
vwndc_ism_7 = np.array(filec_7['V'].loc[:,:,-90:80,60:100])[5:].mean(-1)
vwndh_ism_7 = np.array(fileh_7['V'].loc[:,:,-90:80,60:100])[5:].mean(-1)
vwndc_ism_8 = np.array(filec_8['V'].loc[:,:,-90:80,60:100])[5:].mean(-1)
vwndh_ism_8 = np.array(fileh_8['V'].loc[:,:,-90:80,60:100])[5:].mean(-1)

In [ ]:
hgtc = np.concatenate((hgtc_6,hgtc_7,hgtc_8),axis=0)
hgth = np.concatenate((hgth_6,hgth_7,hgth_8),axis=0)
slpc = np.concatenate((slpc_6,slpc_7,slpc_8),axis=0)
slph = np.concatenate((slph_6,slph_7,slph_8),axis=0)
airc = np.concatenate((airc_6,airc_7,airc_8),axis=0)
airh = np.concatenate((airh_6,airh_7,airh_8),axis=0)
uwndc = np.concatenate((uwndc_6,uwndc_7,uwndc_8),axis=0)
uwndh = np.concatenate((uwndh_6,uwndh_7,uwndh_8),axis=0)
vwndc = np.concatenate((vwndc_6,vwndc_7,vwndc_8),axis=0)
vwndh = np.concatenate((vwndh_6,vwndh_7,vwndh_8),axis=0)
vwndc_ism = np.concatenate((vwndc_ism_6,vwndc_ism_7,vwndc_ism_8),axis=0)
vwndh_ism = np.concatenate((vwndh_ism_6,vwndh_ism_7,vwndh_ism_8),axis=0)

In [ ]:
uwnd_diff, uwnd_p = scp.two_mean_test(np.array(uwndh),np.array(uwndc))
vwnd_diff, vwnd_p = scp.two_mean_test(np.array(vwndh),np.array(vwndc))
hgt_diff, hgt_p = scp.two_mean_test(np.array(hgth),np.array(hgtc))
slp_diff, slp_p = scp.two_mean_test(np.array(slph),np.array(slpc))
air_diff, air_p = scp.two_mean_test(np.array(airh),np.array(airc))
uwnd_copy = np.where((uwnd_p > 0.05) & (vwnd_p > 0.05) , np.nan, uwnd_diff)
vwnd_copy = np.where((uwnd_p > 0.05) & (vwnd_p > 0.05) , np.nan, vwnd_diff)

In [ ]:
def get_cmap_pr(n_colors: int, newcmap , method: int = None) -> list:
    index = list(range(1, n_colors + 1))
    color_list = [newcmap(i / n_colors) for i in index]
    if method == 1:
        color_list[0] = [1., 1., 1.]  
    elif method == 2:
        mid_index = len(color_list) // 2
        color_list[mid_index] = [1., 1., 1.]  
        color_list[mid_index - 1] = [1., 1., 1.]
    return color_list
new_CBR_coldhot = mpl.colors.ListedColormap(get_cmap_pr(18,cmaps.CBR_coldhot,method=2))
new_cmocean_balance_r = mpl.colors.ListedColormap(get_cmap_pr(20,cmaps.cmocean_balance_r,method=2))

In [ ]:
cycle_hgt_diff,cycle_lon = add_cyclic_point(hgt_diff, coord=lon)
cycle_hgt_p,cycle_lon = add_cyclic_point(hgt_p, coord=lon)
proj = ccrs.PlateCarree(central_longitude=150)
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(111,projection = proj)
ax1.draw_map([-60,300,-90,30],latspec=30)
ax1.set_title('GPH@200hPa',fontsize=20,loc='center')
ax1.set_title('a',fontsize=24,loc='left',weight='bold')
cf1 = ax1.contourf(cycle_lon,lat,cycle_hgt_diff,levels=np.arange(-110,120,10), 
    zorder=2, extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
ax1.contourf(cycle_lon,lat,  cycle_hgt_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.5,pad=0.02)
cb.ax.set_colorbar(minor=False)
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/npj_figure/figure-3a.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
cycle_slp_diff,cycle_lon = add_cyclic_point(slp_diff, coord=lon)
cycle_slp_p,cycle_lon = add_cyclic_point(slp_p, coord=lon)
proj = ccrs.PlateCarree(central_longitude=150)
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(111,projection = proj)
ax1.draw_map([-60,300,-90,30],latspec=30)
ax1.set_title('SLP&wind10m',fontsize=20,loc='center')
ax1.set_title('b',fontsize=24,loc='left',weight='bold')
k=6
q = ax1.quiver(lon[::k],lat[::k],(uwnd_diff)[::k,::k],(vwnd_diff)[::k,::k],scale=45,color='gray',alpha=0.9,
               zorder=100,transform=ccrs.PlateCarree())
q = ax1.quiver(lon[::k],lat[::k],uwnd_copy[::k,::k],vwnd_copy[::k,::k],color='k',scale=45,
            zorder=100,transform=ccrs.PlateCarree())
qk = ax1.quiverkey(q, 1.036, 0.99, U=2, label='2',labelpos='N',zorder=3, coordinates='axes',
                   color='k',fontproperties={'size': 18},labelcolor='k')
cf1 = ax1.contourf(cycle_lon,lat,cycle_slp_diff,levels=np.linspace(-600,600,11),
     zorder=2, extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
ax1.contourf(cycle_lon,lat,  cycle_slp_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.5,pad=0.02)
cb.ax.set_colorbar(minor=False)
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/npj_figure/figure-3b.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
theta = np.linspace(0, 2*np.pi, 100)  
center, radius = [0.5, 0.5], 0.5 
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)  
fig = plt.figure(figsize=(5, 4))
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=120.0, central_latitude=-90.0, globe=None))
ax.set_extent([-180, 180, -50, -90], ccrs.PlateCarree())
gl = ax.gridlines(linestyle='--',zorder=2)
ax.set_boundary(circle, transform=ax.transAxes)  # 设置子图边界
ax.add_feature(cfeature.COASTLINE.with_scale('110m'),zorder=3,linewidth=1) 
ax.set_title('SAT&wind10m',fontsize=20,loc='center')
ax.set_title('c',fontsize=24,loc='left',weight='bold')
uwnd_diff[0:10,:],vwnd_diff[0:10,:] = np.nan,np.nan
uwnd_copy[0:10,:],vwnd_copy[0:10,:] = np.nan,np.nan
k=6
q = ax.quiver(lon[::k],lat[::k],(uwnd_diff)[::k,::k],(vwnd_diff)[::k,::k],scale=18,color='gray',alpha=0.9,
               zorder=100,headwidth=5,width=0.004,transform=ccrs.PlateCarree())
q = ax.quiver(lon[::k],lat[::k],uwnd_copy[::k,::k],vwnd_copy[::k,::k],color='k',scale=18,
            zorder=100,headwidth=5,width=0.004,transform=ccrs.PlateCarree())
qk = ax.quiverkey(q, 1.07, 0.92, U=2, label='2',labelpos='N',
    zorder=3, coordinates='axes',color='k',fontproperties={'size': 18},
    labelcolor='k')
cf1 = ax.contourf(lon,lat,air_diff, zorder=2, levels=np.arange(-2.4,2.6,0.6),extend = 'both',transform=ccrs.PlateCarree(), cmap=cmaps.BlueWhiteOrangeRed)
ax.contourf(lon,lat, air_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.7,pad=0.035)
cb.ax.set_colorbar(minor='off')
cb.set_ticks(np.arange(-2.4,3,1.2))
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/npj_figure/figure-3c.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
coslat = np.cos(lat.loc[-90:80].values*np.pi/180)
g = 9.8 
a = 6400000 
kk = lat.loc[-90:80].values.shape[0]
msfc = np.zeros([30,30,kk])
msfh = np.zeros([30,30,kk])

In [ ]:
for t in range(30):
    for j in range(30): 
        for k in range(kk):
            msfc_lev = np.trapz(vwndc_ism[t,0:j,k],lev[0:j],axis=0)
            msfc[t,j,k] = 2*np.pi*a*msfc_lev*coslat[k]/g
            msfh_lev = np.trapz(vwndh_ism[t,0:j,k],lev[0:j],axis=0)
            msfh[t,j,k] = 2*np.pi*a*msfh_lev*coslat[k]/g
msfcu = msfc/(10**9) 
msfhu = msfh/(10**9)
msf_diff, msf_p = scp.two_mean_test(msfhu,msfcu)

In [ ]:
fig = plt.figure(figsize=(16.45,4))
ax1 = plt.subplot(1, 1, 1)
ax1.set_detail()
ax1.axvline(10,c='g',lw=2)
ax1.axvline(28,c='g',lw=2)
ax1.set_ylabel('High (hPa)',fontsize=20)
ax1.set_yscale('symlog')
ax1.set_yticks([200,300,500,700,850,1000])
ax1.set_yticklabels(['200','300','500','700','850','1000'],fontsize=20)
ax1.set_ylim(1000,200) 
ax1.set_xlim(-90,30) 
ax1.set_xticks([-90,-75,-60,-45,-30,-15,0,15,30])
ax1.set_xticklabels([r'90$^\degree$S',r'75$^\degree$S',
    r'60$^\degree$S',r'45$^\degree$S', r'30$^\degree$S', 
    r'15$^\degree$S', r'0$^\degree$',
    r'15$^\degree$N',  r'30$^\degree$N'] ,fontsize=20)
pos = fig.add_axes([0.91,0.125,0.008,0.75])
c1 = ax1.contourf(lat.loc[-90:80],lev,msf_diff,levels=np.linspace(-4,4,11), 
    extend = 'both',zorder=0, cmap=cmaps.MPL_RdBu_r)
ax1.contourf(lat.loc[-90:80],lev, msf_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none")
c2 = ax1.contour(lat.loc[-90:80],lev,msfcu.mean(0),levels=np.linspace(-10,10,21), extend = 'both',zorder=1, colors = 'k',)
cb = fig.colorbar(c1,orientation='vertical',format='%.1f',cax=pos)
cb.ax.set_colorbar(minor='off',fontsize=18)
ax1.set_title("Mass stream function",fontsize=20,loc='center')
ax1.set_title("d",fontsize=24,weight='bold',loc='left')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/npj_figure/figure-3d.jpeg",dpi=600, bbox_inches='tight')